# Gerekli Kütüphanelerin Eklenmesi 

In [1353]:
#Gerekli kütüphaneleri projeye ekleyerek başlıyoruz. 
import pandas as pd #veri işlemleri için pandas kütüphanesini kullanırız
import numpy as np  #sayısal işlemler için numpy kullanacağız
import matplotlib.pyplot as plt #grafik işlemleri için 
from sklearn.model_selection import train_test_split , cross_val_score #test-eğitim verileri bölme vb. işlemler için
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler #veri ölçeklendirme

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR #SVR modeli ---> benim ödevdeki algoritmam 
from sklearn.linear_model import LinearRegression #lineer regresyon 
from sklearn.preprocessing import PolynomialFeatures #Polinom regresyon 
from sklearn.metrics import mean_absolute_error, r2_score #Değerlendirme metrikleri
from sklearn.pipeline import Pipeline

#grafiklerin notebook içinde görünmesini sağlamak için aşağıdaki komutu yazıyoruz
%matplotlib inline 

# Veri Önişleme 

* Veri Setini Okuma Ve İlk 5 Veriyi Yazdırma / Eksik Değerleri Kontrol

In [ ]:
#Verileri okuyacağız 
file_path = "data/forest_fires.csv" #veri setinin yolunu yazıyoruz. 
data = pd.read_csv(file_path, skiprows=1, header=0, encoding='utf-8') #Burada dosyayı okuyoruz. 

print("Veri Setinin Ilk 5 Satırı:") #veri setinde bulunan ilk 5 veriyi çekiyoruz
display(data.head()) #ilk 5 veriyi çekecek komut

print("\nVeri Setinin Bilgisi:")
data.info() #veri seti hakkında genel bilgiyi bize yazacak

print("\nEksik Değerler:")
print(data.isnull().sum()) #eksik değerleri kontol edeceğiz ve print ile yazdıracağız


* Eksik Değerlerin Temizlenmesi/Düzeltilmesi 

In [ ]:
#ilk denemem de veri türlerini dikkate almadığım için hata aldım
#veri türlerini kontrol ederek sütunları inceleyelim 
print("\n Sütunlar ve veri türleri")
print(data.dtypes) #bu komut ile veri türlerini kontrol ediyoruz :)

#veri tablosundaki yazım şeklinden ötürü hata veriyordu. 
#Ben de bunu önlemek adına hepsinin başındaki boşlukları silip hepsini büyük harfle yazdırdım.
data.columns = data.columns.str.strip().str.upper()
# Eksik Değer Kontrol ve Doldurma
numeric_columns = ['TEMPERATURE', 'RH', 'WS', 'RAIN', 'FFMC', 'DMC', 'DC', 'ISI', 'BUI', 'FWI']
categorical_columns = ['DAY', 'CLASSES']

for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())
for col in categorical_columns:
    data[col] = data[col].fillna(data[col].mode()[0])

# Kategorik Verileri Sayısallaştırma
categorical_columns = ['DAY', 'CLASSES']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

#eksik sütunları ortalama alarak tamamlıyoruz
print("\n Eksik değerlerin doldurulması")
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())

print("\n Sütunlar ve veri türleri")
print(data.dtypes)

#Eksik değerler doldurulduktan sonrası
print("\n Sonrası")
print(data.isnull().sum())
print("\n İlk 5 veri")
display(data.head())

* Öznitelik Seçimi 

In [ ]:
#Ödev de bana "ilk 1-6 arası öznitelikler kullanılarak modelleme yapılacaktır " şeklinde bir ibare verildiği için buna dikkate alarak öznitelik seçimlerini yapacağım
# ilk 1-6 sütun bağımsız değişkenler yani x olacak 
x = data.iloc[:, 1:7]
print(x.columns)

#Bağımlı değişken olarak FWI sütununu seçeceğim
y = data['FWI']

#Seçimleri doğrulamak için verilerin boyutunu kontrol edeceğiz
print("\n Bağımsız Değişken Boyutu", x.shape)
print("\n Bağımlı Değişken Boyutu", y.shape )

#ilk 5 satır görüntüleme 
print("\n Bağımsız Değişkenler:")
display(x.head())
print("\n Bağımlı Değişken:")
display(y.head())


# Verinin Eğitim ve Test Setlerine Ayrılması 

In [ ]:
#Veri setini eğitim ve test verisi olarak ikiye böleceğiz. 
#Ben %70 eğitim - %30 test verisi olarak ayıracağım fakat %80 eğitim - %20 test şeklinde de bölünebilir
#Ben projenin performansını ölçmek için 70-30 seçtim. Model daha iyi öğrensin istersek 80-20 seçilebilir.

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=42) #bu komut ile veri setini böldük
print("\nEğitim Seti Boyutu:")
print("x_train:", x_train.shape, "y_train:", y_train.shape) #shape ile görüntüleme işlemi yapacağız
print("\nTest Seti Boyutu:")
print("x_test:", x_test.shape, "y_test:", y_test.shape)


* Veri Ölçeklendirme

In [ ]:
#StandardScaler, veri setindeki özelliklerin(features) standartlaştırılmasını sağlar. 
#sklearn.preprocessing modülünde yer alır
#veriyi hazırlamak için kullanacağız 
#Bunu kullanmak bize 2 katkı sağlayacak
#1. Modelin daha hızlı ve daha doğru öğrenmesi sağlanır.
#2. Özniteliklerin (features) farklı ölçekleri arasında dengesizlik önlenir.

#SVR modelinde veri ölçeklendirme yapacağız. Bunu yaparken StandardScaler kullanarak normalleştirme yapacağız.
#ilk yazdığım kodda sayısal olmayan değişkenleri hesaba katmamıştım. 
#StandardScaler yalnızca sayısal değerler üzerinde çalışır.

#Sayısal sütunlara dönüştürme işlemi yapalım sonrasında NaN değerler için kontrol ve doldurma işlemi yapalım
x_train = x_train.apply(pd.to_numeric, errors='coerce')
x_test = x_test.apply(pd.to_numeric, errors='coerce')
x_train = x_train.fillna(x_train.mean())#fillna() fonksiyonu ile eksik değerler doldurulur
x_test = x_test.fillna(x_test.mean())

scaler = StandardScaler() #Ölçeklendirme işlemine burada başlıyoruz

x_train_scaled = scaler.fit_transform(x_train) #eğitim seti ölçeklendirme
x_test_scaled = scaler.transform(x_test) #test setini ölçeklendirme işlemi

#y_train ölçeklendirme işlemi yapıyoruz
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.values.reshape(-1, 1)).ravel()
y_test_scaled = y_scaler.transform(y_test.values.reshape(-1, 1))


print("\n Ölçeklendirme Eğitim Seti Boyutu", x_train_scaled.shape)
print("\n Ölçeklendirilmiş Test Seti Boyutu:", x_test_scaled.shape)

# Model Kurulumu 
* SVR için model oluşturup eğiteceğiz
* kernel= 'rbf' bu SVR modelinin kullandığı çekirdek fonksiyonunu belirtir.
* rbf(Radial Basis Function), doğrusal olmayan ilişkiler için yaygın olarak kullanılan bir çekirdek fonksiyonudur. Veriler arasındaki mesafeyi hesaplarken daha esnek bir model sağlar. 

** Modeli geliştirmek için hiperparametre optimizasyonu yapacağım. 
SVR için

In [ ]:
svr_model = SVR(kernel='rbf', C=10, epsilon=0.5, gamma=0.1) #modeli tanımlıyoruz / performansını ölçükten sonra  Hiperparametre optimizasyonu denemesi yaptım
svr_model.fit(x_train_scaled, y_train_scaled) #modeli eğitim verisi ile eğitiyoruz
print("\n SVR modeli eğitildi")

In [ ]:

# Hiperparametre Optimizasyonu (SVR)
param_grid_svr = {
    'C': [1, 10, 100],
    'epsilon': [0.1, 0.5, 1],
    'gamma': ['scale', 'auto', 0.1, 1],
    'kernel': ['rbf', 'linear']
}

grid_search_svr = GridSearchCV(SVR(), param_grid_svr, cv=5, scoring='r2', verbose=1)
grid_search_svr.fit(x_train_scaled, y_train_scaled)
best_svr = grid_search_svr.best_estimator_

* PR - Polynomial Regression

** Hiperparametre Optimizasyonu (Polynomial Regression)


In [ ]:

# Pr 
poly_features = PolynomialFeatures(degree = 2) #polinom derecesine verdik
#eğitim ve testi dönüşüme sokuyoruz
x_train_poly = poly_features.fit_transform(x_train_scaled) 
x_test_poly = poly_features.transform(x_test_scaled)

#modeli eğitme 
pr_model = LinearRegression()
pr_model.fit(x_train_poly, y_train_scaled)

In [ ]:
pipeline_pr = Pipeline([
    ('poly_features', PolynomialFeatures()),
    ('linear_regression', LinearRegression())
])
param_grid_pr = {
    'poly_features__degree': [2, 3, 4]
}
grid_search_pr = GridSearchCV(pipeline_pr, param_grid_pr, cv=5, scoring='r2', verbose=1)
grid_search_pr.fit(x_train_scaled, y_train_scaled)
best_pr = grid_search_pr.best_estimator_

# Tahminler Ve Performans Değerlendirmesi
* Test verisi ile tahmin yapacağız
* Metrikleri kullanarak performans değerlendireceğiz
* Sonuçları görselleştireceğiz


* SVR

In [ ]:
y_pred_scaled = best_svr.predict(x_test_scaled) #test verisi ile burada tahmin yapıyoruz
y_pred = y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1)) #tahmin edilen değerleri orijinal ölçeğe dönüştürüyoruz

print("\nGerçek ilk 5 değer: ")
print(y_test[:5].values)

print("\n Tahmin edilen değerler")
print(y_pred[:5].ravel())

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


print("\n Model Performansı:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"R² Skoru: {r2:.4f}")

* PR

In [ ]:
#test ile tahmin yapma
y_pred_pr_scaled = best_pr.predict(x_test_scaled)
y_pred_pr = y_scaler.inverse_transform(y_pred_pr_scaled.reshape(-1, 1))

#performans değerlendirme
mae_pr = mean_absolute_error(y_test, y_pred_pr)
r2_pr = r2_score(y_test, y_pred_pr)

print("\n Pol Reg performans")
print(f"Mean Absolute Error (MAE): {mae_pr:.4f}")
print(f"R² Skoru: {r2_pr:.4f}")

## Çapraz Doğrulama

In [1366]:
cv_scores_svr = cross_val_score(best_svr, x_train_scaled, y_train_scaled, cv=5, scoring='r2')
cv_scores_pr = cross_val_score(best_pr, x_train_scaled, y_train_scaled, cv=5, scoring='r2')

# Görselleştirme 


** Görselleştirme yaparken önce SVR için sonra PR algoritması için grafikler çizdirdim. Hemen ardından Çapraz Doğrulamayı gösterdim. Son olarak da iki algoritmayı ve gerçek değerleri bir grafikte karşılaştırdım. 

In [ ]:
#SVR 1.grafik
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Gerçek Değerler', color='blue')
plt.plot(y_pred, label='Tahmin Edilen Değerler', color='red', linestyle='--')
plt.title("SVR Modeli: Gerçek Değerler vs Tahmin Edilen Değerler")
plt.xlabel("Örnekler")
plt.ylabel("FWI Değeri")
plt.legend()
plt.show()

In [ ]:
#SVR 2. grafik
plt.figure(figsize=(8, 6))
plt.scatter(y_test.values, y_pred, color='blue', edgecolors='k', alpha=0.6)
plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
plt.title("SVR Modeli: Gerçek Değerler vs Tahmin Edilen Değerler")
plt.xlabel("Gerçek Değerler")
plt.ylabel("Tahmin Edilen Değerler")
plt.grid(True)
plt.show()

In [ ]:
# SVR Veri Dağılımı ve Regresyon Doğrusu Grafiği
plt.figure(figsize=(10, 6))

# Veri noktaları (gerçek değerler)
plt.scatter(range(len(y_test)), y_test.values, color='red', label='Gerçek Değerler', alpha=0.6)

# Tahmin edilen eğri (SVR tahminleri)
plt.plot(range(len(y_pred)), y_pred, color='blue', label='SVR Tahmin Eğrisi', linewidth=2)

plt.title("SVR: Veri Dağılımı ve Regresyon Eğrisi")
plt.xlabel("Örnekler")
plt.ylabel("FWI Değeri")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 6. PR Sonuçlarını Görselleştirme
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Gerçek Değerler', color='blue')
plt.plot(y_pred_pr, label='PR Tahminleri', color='green', linestyle='-.')
plt.title("Polynomial Regression: Gerçek Değerler vs Tahmin Edilen Değerler")
plt.xlabel("Örnekler")
plt.ylabel("FWI Değeri")
plt.legend()
plt.show()

In [ ]:
# PR Veri Dağılımı ve Polinom Eğrisi Grafiği
plt.figure(figsize=(10, 6))

# Veri noktaları (gerçek değerler)
plt.scatter(range(len(y_test)), y_test.values, color='red', label='Gerçek Değerler', alpha=0.6)

# Tahmin edilen polinom eğrisi (PR tahminleri)
plt.plot(range(len(y_pred_pr)), y_pred_pr, color='green', label='Polinom Tahmin Eğrisi', linewidth=2)

plt.title("Polynomial Regression: Veri Dağılımı ve Polinom Eğrisi")
plt.xlabel("Örnekler")
plt.ylabel("FWI Değeri")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Çapraz Doğrulama Sonuçları
print("SVR Çapraz Doğrulama Skorları:", cv_scores_svr)
print("SVR Ortalama CV Skoru:", np.mean(cv_scores_svr))
print("Polynomial Regression Çapraz Doğrulama Skorları:", cv_scores_pr)
print("Polynomial Regression Ortalama CV Skoru:", np.mean(cv_scores_pr))
# Sonuçların Karşılaştırılması
results = {
    'Model': ['SVR', 'Polynomial Regression'],
    'R² Score': [r2, r2_pr],
    'MAE': [mae, mae_pr]
}
results_df = pd.DataFrame(results)
print(results_df)



In [ ]:
#iki modelin karşılaştırılması
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Gerçek Değerler', color='blue')
plt.plot(y_pred, label='SVR Tahminleri', color='red', linestyle='--')
plt.plot(y_pred_pr, label='PR Tahminleri', color='green', linestyle='-.')
plt.title("SVR ve PR Modelleri: Gerçek vs Tahmin Edilen Değerler")
plt.xlabel("Örnekler")
plt.ylabel("FWI Değeri")
plt.legend()
plt.show()
